## Data Cleaning

In this notebook, we:

* matched clinical metadata to the corresponding expression matrices for each omics layer,
* removed duplicate samples/features,
* imputed missing values where needed using a KNN imputer,
* and mapped feature identifiers where required (e.g., Ensembl → HGNC, CpG IDs to manifests).

### Import Packages

In [1]:
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
from modules.utils import save_json, load_json 

# to make sure that names match between all datasets,I will create a map for all new terms
word_map = {
    "Gender" : {"male": "Male", "female": "Female", "Male": "Male", "Female": "Female", "F": "Female", "M": "Male",
                "0.0": "Female", "1.0": "Male"
               },
    "Diagnosis": {"AD": "AD", "normal": "Control", "NPH": "NPH", 
                  "DLB": "DLB", "MCI": "MCI", "NC": "Control", "VaD": "VaD",
                  "NVCID": "NVCID", "VCID": "VCID", "HY": "HY", "Control": "Control",
                  "MCI,30": "MCI,30", "Severe": "Severe AD", "Incipient": "Incipient AD", "Moderate": "Moderate AD",
                  "ad": "AD", "pd": "PD", "nc": "Control", "Alzheimer Disease": "AD", "control": "Control", "Alzheimer's Disease": "AD",
                  "Healthy Control": "Control", "alzheimer patient": "AD",
                  "early stage (control)": "Early Stage AD", "late stage" : "Late Stage AD",
                  "Sham": "Control", "Normal Aging": "Control", "Mild cognitive impairment due to Alzheimer's disease": "MCI",
                  "mild cognitive impairment (MCI)": "MCI", "Normal Young": "Control", "Normal Old": "Control",
                  "1.0": "Control", "2.0": "MCI", "3.0": "MCI+", "4.0": "AD", "5.0": "AD+", "6.0": "Other"
                 }
}

### 1. ROSMAP Alzheimer Disease Dataset 
We will analyze the microRNA, transcriptomics and methylation datasets.  The dataset is downloaded from the Synapse database.

These important files are involved:   
* ROSMAP_arraymiRNA.gct file containing processed data,
* ROSMAP_assay_miRNAarray_nanostring_metadata.csv containing metadata information, and   
* ROSMAP_clinical.csv containing clinical data
* ROSMAP_biospecimen_metadata.csv containing mapping between individual ids and specimen ids
* ROSMAP_assay_rnaSeq_metadata.csv containing transcriptomics metadata
* ROSMAP_RNAseq_FPKM_gene.tsv containing FPKM normalized gene expression data
* ROSMAP_assay_methylationArray_metadata.csv containing methylation metadata
* ROSMAP_arrayMethylation_imputed.tsv.gz containing DNA methylation data.

#### 1.1. MicroRNA Data

In [2]:
# load microRNA processed data
miRNA_data = pd.read_csv("../downloads/ROSMAP/ROSMAP_arraymiRNA.gct", sep="\t", skiprows =1)
miRNA_data.columns = miRNA_data.iloc[0,:]
miRNA_data = miRNA_data[1:]
miRNA_data.index = miRNA_data.Genes
miRNA_data.drop(["Genes", "Description"], axis=1, inplace=True) 
miRNA_data.columns.name = None
miRNA_data.index.name = None
miRNA_data = miRNA_data.T
miRNA_data.head()

,hsa-let-7a,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f,hsa-let-7g,hsa-let-7i,hsa-miR-1,hsa-miR-7,...,hcmv-miR-UL70-3p,hcmv-miR-US5-2,hsv1-miR-H1,hsv1-miR-H3,hsv1-miR-H8,hsv2-miR-H4-5p,kshv-miR-K12-2,kshv-miR-K12-4-5p,kshv-miR-K12-9,mcv-miR-M1-5p
DLPFC_115_1A,11.97669819,12.24760639,11.0756382,11.30708799,10.05072859,10.26631412,12.23572496,10.31621672,6.304962147,5.480079682,...,4.311168498,3.912999025,3.741237948,4.493149996,4.816911547,4.248998129,4.811081274,5.406913192,5.432539766,3.968388818
DLPFC_118_1A,12.61141731,11.77855945,11.68882642,11.50804488,9.807690047,11.10449009,12.10201691,10.29556758,6.590821604,6.043393229,...,4.789608242,5.120470569,4.894540714,4.248669682,4.289753733,5.134048484,4.227214189,5.410094924,5.153247734,4.578225851
DLPFC_131_1A,13.34787175,11.44759669,11.79725638,11.37125239,10.57078188,10.76700686,12.19216992,10.10594014,6.906484109,5.695384442,...,4.660273548,4.275072133,4.076803791,4.141251796,4.327639074,3.989854877,3.932535094,4.34253085,4.867532184,5.298846422
DLPFC_137_1A,12.53863398,12.06489349,11.94099212,10.9726096,10.04341679,10.64766318,11.54119663,10.13088351,6.169064638,5.018695507,...,4.38496278,4.897676345,4.235733209,4.810449214,3.556926565,4.469852999,4.104784438,3.915794488,4.11627503,4.919811187
DLPFC_145_1A,12.32396324,12.11557529,11.1352294,11.4697836,10.03414852,10.51152499,11.94412956,10.45428878,6.75810189,4.403832774,...,4.539721111,4.81499839,3.745549917,4.352175099,4.096805726,4.382391595,3.738362546,4.691110297,5.133829992,4.567993746


In [3]:
# check miRNA data shape
miRNA_data.shape

(702, 309)

In [4]:
# load metadata
miRNA_metadata = pd.read_csv("../downloads/ROSMAP/ROSMAP_assay_miRNAarray_nanostring_metadata.csv") 
miRNA_metadata.head()

,specimenID,mirna_id,platform,RIN,rnaBatch,libraryBatch,nanostringBatch,libraryPrep,libraryPreparationMethod,geneRLF,bindingDensity,plate,assay,mirna_id.1
0,253_120426,DLPFC_253_3F,NanoString nCounter miRNA expression array,7.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,mirnaArray,DLPFC_253_3F
1,R02_131017,NaN,NanoString nCounter miRNA expression array,4.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,mirnaArray,NaN
2,503_120515,NaN,NanoString nCounter miRNA expression array,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,mirnaArray,NaN
3,520_120515,DLPFC_520_6C,NanoString nCounter miRNA expression array,8.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,mirnaArray,DLPFC_520_6C
4,691_120605,DLPFC_691_8D,NanoString nCounter miRNA expression array,6.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,mirnaArray,DLPFC_691_8D


In [5]:
# load clinical data
clinical_data = pd.read_csv("../downloads/ROSMAP/ROSMAP_clinical.csv") 
clinical_data.head()

,projid,Study,msex,educ,race,spanish,apoe_genotype,age_at_visit_max,age_first_ad_dx,age_death,cts_mmse30_first_ad_dx,cts_mmse30_lv,pmi,braaksc,ceradsc,cogdx,dcfdx_lv,individualID
0,10101589,ROS,1.0,20.0,1.0,2.0,34.0,90+,90+,90+,18.0,5.0,9.916667,4.0,2.0,4.0,4.0,R6939144
1,86767530,MAP,0.0,10.0,1.0,2.0,33.0,90+,90+,90+,18.0,10.0,6.500000,4.0,2.0,4.0,4.0,R3893503
2,9650662,MAP,0.0,15.0,1.0,2.0,23.0,90+,90+,90+,0.0,0.0,3.850000,3.0,2.0,4.0,4.0,R8937093
3,50402855,MAP,0.0,21.0,1.0,2.0,33.0,90+,NaN,NaN,NaN,27.0,NaN,NaN,NaN,NaN,1.0,R7139444
4,20544321,ROS,0.0,16.0,1.0,2.0,23.0,90+,90+,NaN,13.0,14.0,NaN,NaN,NaN,NaN,4.0,R4971237


In [6]:
# count class distribution
clinical_data.cogdx.value_counts()

cogdx
4.0    674
1.0    586
2.0    404
5.0     94
3.0     33
6.0     30
Name: count, dtype: int64

In [7]:
# load ROSMAP_biospecimen_metadata
biospecimen_metadata = pd.read_csv("../downloads/ROSMAP/ROSMAP_biospecimen_metadata.csv") 
biospecimen_metadata.head()

,individualID,specimenID,specimenIdSource,organ,tissue,BrodmannArea,sampleStatus,tissueWeight,tissueVolume,nucleicAcidSource,cellType,fastingState,isPostMortem,samplingAge,samplingAgeUnits,visitNumber,assay,exclude,excludeReason,samplingDate
0,R1743384,190403-B4-A_R1743384,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,False,NaN,NaN
1,R2670295,190403-B4-A_R2670295,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,False,NaN,NaN
2,R4119160,190403-B4-A_R4119160,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,True,RNA genotype discordant with WGS,NaN
3,R4641987,190403-B4-A_R4641987,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,False,NaN,NaN
4,R5693901,190403-B4-A_R5693901,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,True,Duplicated donor,NaN


**Observation**  
* The sepcimen column in the miRNA metadata may be related to the individual id column in the clinical data.
* The mirna_id column in the metadata may be used to match sample names in the expression data
* clinical data column features are explained in the clinical data cookbook.
* The biospeciment column can be used to map specimen ids to individual ids. 

In [8]:
# create specimen_to_individual_id map
df = biospecimen_metadata.iloc[:,0:2]
df.dropna(inplace=True)
specimenID_to_individualID_map = dict(zip(df.specimenID, df.individualID))

In [9]:
# create sample_id to specimen id map
df2 = miRNA_metadata.iloc[:,0:2]
df2.dropna(inplace = True)
sampleID_to_specimenID_map = dict(zip(df2.mirna_id, df2.specimenID))

In [10]:
# How many known sample ids are there
print("Number of known samples: ", len(set(miRNA_data.index)))

Number of known samples:  702


In [11]:
# How many samples have mapped speciment ids
mapped_samples = [sample_id for sample_id in miRNA_data.index if sample_id in sampleID_to_specimenID_map]

print("Number of mapped sample_ids to specimen ids: ", len(set(mapped_samples)))

Number of mapped sample_ids to specimen ids:  525


In [12]:
# How many mapped specimen ids have mapped individual ids?
mapped_specimens = [sampleID_to_specimenID_map[sample_id] for sample_id in mapped_samples \
                    if sampleID_to_specimenID_map[sample_id] in specimenID_to_individualID_map]

print("Number of sample ids with individual id mappings: ", len(mapped_specimens))

Number of sample ids with individual id mappings:  525


In [13]:
# fecth the sample ids that have individualid mapping
mappable_sampleIDs = [sample_id for sample_id in miRNA_data.index \
                     if sample_id in mapped_samples and sampleID_to_specimenID_map[sample_id] in mapped_specimens
                     ]

len(mappable_sampleIDs)

525

In [14]:
# map expression data indices
miRNA_data = miRNA_data.loc[mappable_sampleIDs]
miRNA_data.index = [specimenID_to_individualID_map[sampleID_to_specimenID_map[sample_id]] \
                        for sample_id in miRNA_data.index]
miRNA_data.head()

,hsa-let-7a,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,hsa-let-7f,hsa-let-7g,hsa-let-7i,hsa-miR-1,hsa-miR-7,...,hcmv-miR-UL70-3p,hcmv-miR-US5-2,hsv1-miR-H1,hsv1-miR-H3,hsv1-miR-H8,hsv2-miR-H4-5p,kshv-miR-K12-2,kshv-miR-K12-4-5p,kshv-miR-K12-9,mcv-miR-M1-5p
R4620822,11.97669819,12.24760639,11.0756382,11.30708799,10.05072859,10.26631412,12.23572496,10.31621672,6.304962147,5.480079682,...,4.311168498,3.912999025,3.741237948,4.493149996,4.816911547,4.248998129,4.811081274,5.406913192,5.432539766,3.968388818
R6057469,12.61141731,11.77855945,11.68882642,11.50804488,9.807690047,11.10449009,12.10201691,10.29556758,6.590821604,6.043393229,...,4.789608242,5.120470569,4.894540714,4.248669682,4.289753733,5.134048484,4.227214189,5.410094924,5.153247734,4.578225851
R2703808,13.34787175,11.44759669,11.79725638,11.37125239,10.57078188,10.76700686,12.19216992,10.10594014,6.906484109,5.695384442,...,4.660273548,4.275072133,4.076803791,4.141251796,4.327639074,3.989854877,3.932535094,4.34253085,4.867532184,5.298846422
R2716798,12.32396324,12.11557529,11.1352294,11.4697836,10.03414852,10.51152499,11.94412956,10.45428878,6.75810189,4.403832774,...,4.539721111,4.81499839,3.745549917,4.352175099,4.096805726,4.382391595,3.738362546,4.691110297,5.133829992,4.567993746
R4174623,12.41298829,11.29822783,11.15170767,11.07019661,10.21787742,10.62930529,11.95181589,9.771981065,6.617407203,6.03704994,...,4.255314615,4.741535048,4.185454727,3.884882111,4.566701561,4.354368214,4.322092931,5.371338061,5.372921051,4.469409903


In [15]:
# filter clinical data by common individual ids
clinical_data.index = clinical_data.individualID
clinical_data = clinical_data.loc[miRNA_data.index]
clinical_data.head()

,projid,Study,msex,educ,race,spanish,apoe_genotype,age_at_visit_max,age_first_ad_dx,age_death,cts_mmse30_first_ad_dx,cts_mmse30_lv,pmi,braaksc,ceradsc,cogdx,dcfdx_lv,individualID
R4620822,20264936,ROS,0.0,20.0,1.0,2.0,33.0,87.701574264202605,NaN,88.65708418891171,NaN,29.0,3.583333,3.0,2.0,2.0,2.0,R4620822
R6057469,50105725,MAP,1.0,17.0,1.0,2.0,23.0,86.617385352498289,NaN,87.466119096509246,NaN,27.0,21.633333,4.0,3.0,2.0,3.0,R6057469
R2703808,20730959,ROS,0.0,16.0,1.0,2.0,23.0,84.208076659822041,NaN,85.163586584531146,NaN,28.0,3.250000,3.0,2.0,1.0,1.0,R2703808
R2716798,20151388,ROS,0.0,16.0,1.0,2.0,33.0,83.030800821355243,79.991786447638603,83.266255989048602,27.0,16.0,5.416667,5.0,1.0,4.0,4.0,R2716798
R4174623,11259716,ROS,1.0,10.0,1.0,2.0,33.0,90+,NaN,90+,NaN,28.0,6.000000,3.0,2.0,1.0,1.0,R4174623


**Now let's clean the individual data**  
Columns to select:
* msex: Gender
* race: Race
* pmi: Post-mortem interval
* braaksc: Braak Stage
* cogdx:  Diagnosis

In [16]:
# clean clinical data
clinical_data = clinical_data[["msex", "race", "pmi", "braaksc", "cogdx"]]
clinical_data.columns = ["Gender", "Race", "PMI", "Braak", "Diagnosis"]

clinical_data["Race"] = clinical_data["Race"].astype(int)
clinical_data["Gender"] = clinical_data["Gender"].apply(lambda x: word_map["Gender"][str(x)])
clinical_data["Diagnosis"] = clinical_data["Diagnosis"].apply(lambda x: word_map["Diagnosis"][str(x)])
clinical_data["Braak"] = clinical_data["Braak"].astype(int)
clinical_data.head()

,Gender,Race,PMI,Braak,Diagnosis
R4620822,Female,1,3.583333,3,MCI
R6057469,Male,1,21.633333,4,MCI
R2703808,Female,1,3.250000,3,Control
R2716798,Female,1,5.416667,5,AD
R4174623,Male,1,6.000000,3,Control


In [17]:
clinical_data["Diagnosis"] = clinical_data.Diagnosis.apply(lambda x: "AD" if "AD" in x else x) 
clinical_data = clinical_data[clinical_data.Diagnosis.isin(["AD","Control"])] 
clinical_data.shape

(378, 5)

In [31]:
# now merge clinical and expression data
miRNA_data = pd.merge(clinical_data.reset_index(), miRNA_data.reset_index())
miRNA_data.index = miRNA_data["index"]
miRNA_data.drop("index", axis =1, inplace=True)
miRNA_data.index.rename('Sample ID', inplace=True)

In [32]:
# select alzheimer samples
miRNA_data["Diagnosis"] = miRNA_data.Diagnosis.apply(lambda x: "AD" if "AD" in x else x) 
miRNA_data = miRNA_data[miRNA_data.Diagnosis.isin(["AD","Control"])] 
miRNA_data.head()

,Gender,Race,PMI,Braak,Diagnosis,hsa-let-7a,hsa-let-7b,hsa-let-7c,hsa-let-7d,hsa-let-7e,...,hcmv-miR-UL70-3p,hcmv-miR-US5-2,hsv1-miR-H1,hsv1-miR-H3,hsv1-miR-H8,hsv2-miR-H4-5p,kshv-miR-K12-2,kshv-miR-K12-4-5p,kshv-miR-K12-9,mcv-miR-M1-5p
Sample ID,,,,,,,,,,,,,,,,,,,,,
R2703808,Female,1,3.250000,3,Control,13.34787175,11.44759669,11.79725638,11.37125239,10.57078188,...,4.660273548,4.275072133,4.076803791,4.141251796,4.327639074,3.989854877,3.932535094,4.34253085,4.867532184,5.298846422
R2716798,Female,1,5.416667,5,AD,12.32396324,12.11557529,11.1352294,11.4697836,10.03414852,...,4.539721111,4.81499839,3.745549917,4.352175099,4.096805726,4.382391595,3.738362546,4.691110297,5.133829992,4.567993746
R4174623,Male,1,6.000000,3,Control,12.41298829,11.29822783,11.15170767,11.07019661,10.21787742,...,4.255314615,4.741535048,4.185454727,3.884882111,4.566701561,4.354368214,4.322092931,5.371338061,5.372921051,4.469409903
R6681836,Male,1,4.666667,2,AD,12.08814977,11.96855561,11.16850093,11.3348563,9.91211075,...,4.268604152,4.701867689,3.98283835,3.887396327,3.942830277,4.531838368,3.598978977,4.667987688,5.05202363,5.28103257
R3198654,Female,1,11.500000,4,AD,11.94278062,11.56964742,10.51026369,11.75530327,10.04856422,...,4.224442415,5.00635407,3.589346281,4.02883335,4.623907447,3.722625758,4.219962826,5.138899152,5.600866018,3.947512686


In [33]:
# check data shape
miRNA_data.shape

(378, 314)

#### 1.2. Transcriptomics

In [34]:
# load metadata
rnaseq_metadata = pd.read_csv("../downloads/ROSMAP/ROSMAP_biospecimen_metadata.csv")
rnaseq_metadata.head()

,individualID,specimenID,specimenIdSource,organ,tissue,BrodmannArea,sampleStatus,tissueWeight,tissueVolume,nucleicAcidSource,cellType,fastingState,isPostMortem,samplingAge,samplingAgeUnits,visitNumber,assay,exclude,excludeReason,samplingDate
0,R1743384,190403-B4-A_R1743384,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,False,NaN,NaN
1,R2670295,190403-B4-A_R2670295,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,False,NaN,NaN
2,R4119160,190403-B4-A_R4119160,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,True,RNA genotype discordant with WGS,NaN
3,R4641987,190403-B4-A_R4641987,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,False,NaN,NaN
4,R5693901,190403-B4-A_R5693901,NaN,brain,dorsolateral prefrontal cortex,NaN,frozen,NaN,NaN,single nucleus,NaN,NaN,True,NaN,NaN,NaN,scrnaSeq,True,Duplicated donor,NaN


In [35]:
# Load gene expression data
gene_expression = pd.read_csv("../downloads/ROSMAP/ROSMAP_RNAseq_FPKM_gene.tsv", sep = "\t")
rnaseq_sample_names = gene_expression.columns[2:]
gene_expression = gene_expression.iloc[:,1:]
gene_expression.index = gene_expression.gene_id
gene_expression = gene_expression.drop("gene_id", axis=1).T
gene_expression.head()

gene_id,ENSG00000167578.11,ENSG00000242268.1,ENSG00000078237.4,ENSG00000263642.1,ENSG00000225275.4,ENSG00000060642.6,ENSG00000201788.1,ENSG00000263089.1,ENSG00000172137.14,ENSG00000240423.1,...,ENSG00000232668.1,ENSG00000089177.13,ENSG00000216352.1,ENSG00000267117.1,ENSG00000148943.7,ENSG00000265520.1,ENSG00000231119.2,ENSG00000105063.14,ENSG00000123685.4,ENSG00000181518.2
525_120515_0,60.84,0.08,4.39,0.0,0.0,5.98,0.0,0.0,19.69,0.09,...,0.0,4.98,0.0,0.0,10.02,0.0,0.20,40.59,4.46,0.00
383_120503_0,65.45,0.05,4.49,0.0,0.0,4.66,0.0,0.0,31.08,0.18,...,0.0,5.33,0.0,0.0,8.76,0.0,0.20,39.26,4.51,0.00
93_120417_0,69.18,0.08,2.51,0.0,0.0,2.77,0.0,0.0,10.76,0.12,...,0.0,6.95,0.0,0.0,6.28,0.0,0.24,34.70,5.27,0.05
610_120523_0,51.60,0.08,2.90,0.0,0.0,4.50,0.0,0.0,9.14,0.18,...,0.0,5.31,0.0,0.0,8.32,0.0,0.21,36.74,5.11,0.05
560_120517_0,48.61,0.10,2.67,0.0,0.0,4.26,0.0,0.0,5.87,0.17,...,0.0,7.12,0.0,0.0,8.68,0.0,0.09,39.85,3.77,0.00


**Observation**  
Upon examining the dataset, the clinical data contains individual IDs, which match the `individualID` column in the metadata. However, the sample IDs in the gene expression data are similar to the `specimenID` in the metadata, except that the gene expression sample names include an additional "_{Number}" ranging from 1 to 8. We will remove these numbers and map the remaining ID stem accordingly.

In [36]:
# create a specimenID to individualID map
specimenID_to_individualID = dict(zip(rnaseq_metadata.specimenID, rnaseq_metadata.individualID))

In [37]:
# fix gene expression specimenIDs
gene_expression.index = list(map( lambda x: "_".join(x.split("_")[:-1]), gene_expression.index))
gene_expression.head()

gene_id,ENSG00000167578.11,ENSG00000242268.1,ENSG00000078237.4,ENSG00000263642.1,ENSG00000225275.4,ENSG00000060642.6,ENSG00000201788.1,ENSG00000263089.1,ENSG00000172137.14,ENSG00000240423.1,...,ENSG00000232668.1,ENSG00000089177.13,ENSG00000216352.1,ENSG00000267117.1,ENSG00000148943.7,ENSG00000265520.1,ENSG00000231119.2,ENSG00000105063.14,ENSG00000123685.4,ENSG00000181518.2
525_120515,60.84,0.08,4.39,0.0,0.0,5.98,0.0,0.0,19.69,0.09,...,0.0,4.98,0.0,0.0,10.02,0.0,0.20,40.59,4.46,0.00
383_120503,65.45,0.05,4.49,0.0,0.0,4.66,0.0,0.0,31.08,0.18,...,0.0,5.33,0.0,0.0,8.76,0.0,0.20,39.26,4.51,0.00
93_120417,69.18,0.08,2.51,0.0,0.0,2.77,0.0,0.0,10.76,0.12,...,0.0,6.95,0.0,0.0,6.28,0.0,0.24,34.70,5.27,0.05
610_120523,51.60,0.08,2.90,0.0,0.0,4.50,0.0,0.0,9.14,0.18,...,0.0,5.31,0.0,0.0,8.32,0.0,0.21,36.74,5.11,0.05
560_120517,48.61,0.10,2.67,0.0,0.0,4.26,0.0,0.0,5.87,0.17,...,0.0,7.12,0.0,0.0,8.68,0.0,0.09,39.85,3.77,0.00


In [38]:
# print gene expression shape
gene_expression.shape

(640, 55889)

In [39]:
# check how many gene expression specimenIDs can be mapped to individualIDs
mappable_specimenIDs = [specimenID for specimenID in gene_expression.index if specimenID in specimenID_to_individualID]
len(mappable_specimenIDs)

640

In [40]:
# map gene expression data specimenIDs to individualIDs
gene_expression = gene_expression.loc[mappable_specimenIDs]
gene_expression.index = list(map( lambda x: specimenID_to_individualID[x], gene_expression.index))
gene_expression.columns.name = None
gene_expression.head()

,ENSG00000167578.11,ENSG00000242268.1,ENSG00000078237.4,ENSG00000263642.1,ENSG00000225275.4,ENSG00000060642.6,ENSG00000201788.1,ENSG00000263089.1,ENSG00000172137.14,ENSG00000240423.1,...,ENSG00000232668.1,ENSG00000089177.13,ENSG00000216352.1,ENSG00000267117.1,ENSG00000148943.7,ENSG00000265520.1,ENSG00000231119.2,ENSG00000105063.14,ENSG00000123685.4,ENSG00000181518.2
R1743384,60.84,0.08,4.39,0.0,0.0,5.98,0.0,0.0,19.69,0.09,...,0.0,4.98,0.0,0.0,10.02,0.0,0.20,40.59,4.46,0.00
R6862468,65.45,0.05,4.49,0.0,0.0,4.66,0.0,0.0,31.08,0.18,...,0.0,5.33,0.0,0.0,8.76,0.0,0.20,39.26,4.51,0.00
R5415701,69.18,0.08,2.51,0.0,0.0,2.77,0.0,0.0,10.76,0.12,...,0.0,6.95,0.0,0.0,6.28,0.0,0.24,34.70,5.27,0.05
R1407047,51.60,0.08,2.90,0.0,0.0,4.50,0.0,0.0,9.14,0.18,...,0.0,5.31,0.0,0.0,8.32,0.0,0.21,36.74,5.11,0.05
R2197944,48.61,0.10,2.67,0.0,0.0,4.26,0.0,0.0,5.87,0.17,...,0.0,7.12,0.0,0.0,8.68,0.0,0.09,39.85,3.77,0.00


In [41]:
# check duplicates
gene_expression.duplicated().sum()

6

In [42]:
# drop duplicates
gene_expression = gene_expression.drop_duplicates()
gene_expression.duplicated().sum()

0

In [43]:
# check if some individualIDs are nans
gene_expression.index.isna().sum()

2

In [44]:
# remove observations where the the individual ID is missing
gene_expression = gene_expression[gene_expression.index.notna()] 
gene_expression.columns = [column.split(".")[0] for column in gene_expression.columns]
gene_expression.shape

(638, 55889)

In [45]:
import mygene
mg = mygene.MyGeneInfo()
ensembl_ids = [id.split(".")[0] for id in gene_expression.columns]

In [48]:
# Query the Ensembl IDs
results = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='human')

In [ ]:
gene_symbols = []
gene_id_to_symbol = {}
# Print the mapping from Ensembl IDs to gene symbols
for result in results:
    ensembl_id = result.get('query')
    gene_symbol = result.get('symbol', 'N/A')
    gene_id_to_symbol[ensembl_id] = gene_symbol

gene_symbols = [gene_id_to_symbol[gene_id] for gene_id in ensembl_ids]

In [ ]:
# mapped ids
mapped_ids = [gene_id for gene_id in gene_expression.columns if gene_id_to_symbol[gene_id.split(".")[0]] != "N/A" ]
len(mapped_ids)

In [ ]:
# filter gene_expression by mapped genes
gene_expression = gene_expression[mapped_ids]

In [ ]:
# map columns 
gene_expression.columns = [gene_id_to_symbol[gene_id.split(".")[0]] for gene_id in gene_expression.columns] 
gene_expression.head()

In [ ]:
# Remove duplicates while maintaining the original order
new_columns_unique = list(dict.fromkeys(gene_expression.columns))
len(new_columns_unique)

In [ ]:
gene_expression = gene_expression[new_columns_unique]
gene_expression = gene_expression.loc[:, ~gene_expression.columns.duplicated()]
gene_expression.head()

In [ ]:
# Remove columns where the sum is 0
gene_expression = gene_expression.loc[:, (gene_expression.sum() != 0)]
gene_expression.head()

In [ ]:
# check for missing values
gene_expression.isna().sum().sum()

In [ ]:
gene_expression

#### 1.3. Methylation Data

In [41]:
meth_metadata = pd.read_csv("../downloads/ROSMAP/ROSMAP_assay_methylationArray_metadata.csv") 
meth_metadata.head()

,specimenID,platform,dnaBatch,arrayBatch,260/280,260/230,GQN,Sentrix_ID,Sentrix_Row_Column,Sample_Plate,Sample_Well,batch
0,TBI-AUTO73325-PT-3149,IlluminaHumanMethylation450,NaN,NaN,NaN,NaN,NaN,5815381027,R06C01,WG0001328-MSA4,B03,0
1,TBI-AUTO73307-PT-314I,IlluminaHumanMethylation450,NaN,NaN,NaN,NaN,NaN,5815381028,R05C02,WG0001340-MSA4,C08,1
2,TBI-AUTO73043-PT-35BD,IlluminaHumanMethylation450,NaN,NaN,NaN,NaN,NaN,5822038007,R06C01,WG0001338-MSA4,F01,1
3,PT-318X,IlluminaHumanMethylation450,NaN,NaN,NaN,NaN,NaN,6042324057,R02C02,WG0003259-MSA4,H01,1
4,TBI-AUTO73279-PT-35NK,IlluminaHumanMethylation450,NaN,NaN,NaN,NaN,NaN,5822038007,R06C02,WG0001338-MSA4,D02,1


In [42]:
meth_data = pd.read_csv("../downloads/ROSMAP/ROSMAP_arrayMethylation_imputed.tsv.gz", sep = "\t") 
meth_data.head()

,TargetID,TBI-AUTO73325-PT-3149,PT-BZHL,PT-BZCH,PT-BY9H,TBI-AUTO73307-PT-314I,TBI-AUTO73043-PT-35BD,PT-BZI5,PT-BZ1A,PT-318X,...,TBI-AUTO72955-PT-35OC,TBI-AUTO73291-PT-35OD,PT-BZD7,PT-BZG8,PT-C1N8,PT-BYJP,PT-BZHV,PT-BZI2,TBI-AUTO73257-PT-35OE,PT-BZD3
0,cg00000165,0.231359,0.157857,0.127105,0.149988,0.130532,0.174451,0.170026,0.165900,0.157745,...,0.124635,0.168002,0.152200,0.230482,0.177287,0.195611,0.151338,0.151508,0.167960,0.136220
1,cg00000363,0.140664,0.114399,0.140580,0.145805,0.122833,0.117144,0.155559,0.131309,0.157749,...,0.129301,0.126910,0.135973,0.162883,0.122399,0.112684,0.125054,0.118550,0.114618,0.133814
2,cg00000957,0.776220,0.818279,0.630316,0.849261,0.861136,0.751543,0.778917,0.859892,0.787279,...,0.735520,0.740686,0.748494,0.797072,0.793081,0.692638,0.785597,0.712386,0.839168,0.888304
3,cg00001349,0.865488,0.919570,0.882256,0.902912,0.907610,0.897496,0.939212,0.919514,0.805626,...,0.885292,0.900684,0.906257,0.756173,0.948341,0.904996,0.927201,0.856637,0.917827,0.943320
4,cg00001364,0.772899,0.711099,0.694639,0.651986,0.748538,0.706625,0.743491,0.712513,0.722169,...,0.720420,0.724226,0.694462,0.776165,0.758569,0.733207,0.750224,0.737402,0.685959,0.770543


In [43]:
meth_data.set_index("TargetID", inplace = True)
meth_data = meth_data.T 
meth_data.head()

TargetID,cg00000165,cg00000363,cg00000957,cg00001349,cg00001364,cg00001446,cg00001534,cg00001583,cg00001593,cg00002028,...,ch.22.31817810F,ch.22.33863861F,ch.22.533187F,ch.22.740407F,ch.22.757911F,ch.22.772318F,ch.22.43177094F,ch.22.909671F,ch.22.46830341F,ch.22.1008279F
TBI-AUTO73325-PT-3149,0.231359,0.140664,0.776220,0.865488,0.772899,0.842025,0.883920,0.099888,0.918081,0.026821,...,0.161798,0.257386,0.128748,0.139123,0.089335,0.233366,0.173744,0.409177,0.225070,0.067878
PT-BZHL,0.157857,0.114399,0.818279,0.919570,0.711099,0.831457,0.877727,0.076240,0.955305,0.035839,...,0.198222,0.292629,0.125090,0.127840,0.089504,0.198370,0.199095,0.463476,0.279521,0.072976
PT-BZCH,0.127105,0.140580,0.630316,0.882256,0.694639,0.835075,0.843842,0.075861,0.896318,0.020016,...,0.134829,0.256428,0.095440,0.118760,0.059411,0.176775,0.189069,0.307656,0.224084,0.090009
PT-BY9H,0.149988,0.145805,0.849261,0.902912,0.651986,0.834157,0.917063,0.101837,0.941423,0.014101,...,0.145478,0.330483,0.119385,0.091408,0.074204,0.141996,0.226845,0.302488,0.286071,0.051107
TBI-AUTO73307-PT-314I,0.130532,0.122833,0.861136,0.907610,0.748538,0.839395,0.863272,0.071947,0.940628,0.031390,...,0.227004,0.291872,0.120238,0.108485,0.062730,0.167610,0.193875,0.426867,0.272743,0.072089


In [44]:
# map specimen ids
mappable_meth_samples = list(set(meth_data.index).intersection(biospecimen_metadata.specimenID))
len(mappable_meth_samples)

740

In [45]:
# filter samples then map
meth_data = meth_data.loc[mappable_meth_samples]
meth_data.index = [specimenID_to_individualID_map[name] for name in meth_data.index]
meth_data.head()

TargetID,cg00000165,cg00000363,cg00000957,cg00001349,cg00001364,cg00001446,cg00001534,cg00001583,cg00001593,cg00002028,...,ch.22.31817810F,ch.22.33863861F,ch.22.533187F,ch.22.740407F,ch.22.757911F,ch.22.772318F,ch.22.43177094F,ch.22.909671F,ch.22.46830341F,ch.22.1008279F
R2906309,0.189433,0.128630,0.813671,0.945957,0.745644,0.854985,0.902876,0.039295,0.917115,0.019719,...,0.128549,0.281209,0.103040,0.101263,0.108405,0.179309,0.171076,0.311554,0.233715,0.050103
R8330118,0.172273,0.117854,0.738078,0.911470,0.657762,0.852372,0.851241,0.065656,0.942779,0.020461,...,0.137351,0.266473,0.106335,0.117186,0.070021,0.173022,0.193154,0.413421,0.222522,0.037462
R3368249,0.127003,0.137272,0.664511,0.793936,0.677069,0.823069,0.838775,0.072063,0.927902,0.011286,...,0.170992,0.298454,0.150157,0.119495,0.112286,0.168502,0.214770,0.450181,0.254606,0.080463
R4345802,0.160082,0.127053,0.712104,0.915619,0.743355,0.852362,0.863584,0.074825,0.881600,0.016483,...,0.173248,0.294833,0.114080,0.131991,0.058505,0.151754,0.191676,0.479210,0.218384,0.064807
R4361022,0.195934,0.115724,0.757739,0.805134,0.667714,0.832603,0.872678,0.097961,0.927149,0.023921,...,0.169757,0.250568,0.110982,0.126584,0.077510,0.215209,0.171116,0.357705,0.242189,0.064497


In [46]:
# check for missing values
meth_data.isna().sum().sum()

0

#### 1.4 Select Multi-Omics Samples and Save

In [47]:
## select multi-omics samples
set1 = set(miRNA_data.index.to_list()) 
set2 = set(gene_expression.index.to_list())
set3 = set(meth_data.index.to_list())

selected_samples = set1 & set2 & set3 
len(selected_samples)

375

In [48]:
# save datasets
gene_expression_data = pd.merge(miRNA_data[miRNA_data.columns[:5]].reset_index(), gene_expression.reset_index().rename(columns = {"index": "Sample ID"})) 
dna_methylation_data = pd.merge(miRNA_data[miRNA_data.columns[:5]].reset_index(), meth_data.reset_index().rename(columns = {"index": "Sample ID"})) 

miRNA_data.to_csv("../data/ROSMAP/cleaned/miRNA_data.csv")
gene_expression_data.to_csv("../data/ROSMAP/cleaned/gene_expression_data.csv", index = False)
dna_methylation_data.to_csv("../data/ROSMAP/cleaned/dna_methylation_data.csv", index = False)

### 2. MayoRNAseq Dataset (Progressive Supranuclear Palsy)

We will analyze Gene Expression, Proteomics, Metabolomics data. The dataset is downloaded from the Synapse database.

These important files are involved:   
* MayoRNAseq_biospecimen_metadata.csv
* MayoRNAseq_individual_metadata.csv
* Mayo_Proteomics_ID_key.csv
* mayo_all_metabolon_raw.XLSX
* Mayo_Proteomics_TC_proteinoutput.txt
* MayoRNAseq_RNAseq_CBE_geneCounts.tsv
* MayoRNAseq_RNAseq_CBE_geneCounts_normalized.tsv
* MayoRNAseq_RNAseq_TCX_geneCounts.tsv
* MayoRNAseq_RNAseq_TCX_geneCounts_normalized.tsv

In [18]:
# load and clean biospecimen data
mayo_biospecimen = pd.read_csv("../downloads/MayoRNASeq/MayoRNAseq_biospecimen_metadata.csv")
mayo_biospecimen = mayo_biospecimen[mayo_biospecimen.columns[:2]]
mayo_biospecimen = mayo_biospecimen.dropna()
mayo_biospecimen.individualID = mayo_biospecimen.individualID.astype(int)
specimenID_to_individualID_MAYO = dict(zip(mayo_biospecimen.specimenID, mayo_biospecimen.individualID))
mayo_biospecimen.head()

,individualID,specimenID
0,1005,1005_TCX
1,1010,1010_TCX
2,1015,1015_TCX
3,1019,1019_TCX
4,1029,1029_TCX


In [22]:
# load clinical data
mayo_clinical_data = pd.read_csv("../downloads/MayoRNASeq/MayoRNAseq_individual_metadata.csv") 
mayo_clinical_data = mayo_clinical_data[["individualID", "sex", "ageDeath", "Braak", "diagnosis"]] 
mayo_clinical_data = mayo_clinical_data[mayo_clinical_data.diagnosis.isin(['progressive supranuclear palsy', "control"])]
mayo_clinical_data.set_index("individualID", inplace = True)
mayo_clinical_data.index.name = None
mayo_clinical_data["diagnosis"] = mayo_clinical_data["diagnosis"].apply(lambda x: "Control" if x=="control" else "PSP")
mayo_clinical_data.rename(columns = {"sex": "Gender", "diagnosis": "Diagnosis"}, inplace = True)
mayo_clinical_data.head()

,Gender,ageDeath,Braak,Diagnosis
1933,female,90_or_over,1.0,Control
1938,female,90_or_over,2.5,Control
1963,male,90_or_over,3.0,Control
1964,male,90_or_over,3.0,Control
1955,female,87,1.0,Control


In [24]:
mayo_clinical_data = mayo_clinical_data[mayo_clinical_data.Diagnosis.isin(["PSP","Control"])] 
mayo_clinical_data.shape

(437, 4)

In [52]:
# load protein keys
mayo_proteomics_keys = pd.read_csv("../downloads/MayoRNASeq/Mayo_Proteomics_ID_key.csv")
mayo_proteomics_keys = mayo_proteomics_keys[mayo_proteomics_keys.columns[:2]].dropna()
MayKeysMap = dict(zip(mayo_proteomics_keys.Samples_Simple, mayo_proteomics_keys.RNA_SampleID))

#### 2.1. Proteomics Data

In [53]:
import re 
import numpy as np
pat = re.compile(r'(?:^|;)(?:sp|tr)\|[^|]+\|([^;|]+)')  # capture entry name after sp|acc|ENTRY

def extract_uniprot_entry(cell: str) -> str | None:
    if not isinstance(cell, str): 
        return None
    m = pat.search(cell)
    return m.group(1) if m else None

In [60]:
mayo_proteomics = pd.read_csv("../downloads/MayoRNASeq/Mayo_Proteomics_TC_proteinoutput.txt", sep = "\t") 
mayo_proteomics = mayo_proteomics.iloc[:, [0]].join(mayo_proteomics.iloc[:, 10:-15])  
mayo_proteomics["Protein IDs"] = mayo_proteomics["Protein IDs"].apply(extract_uniprot_entry)
mayo_proteomics = mayo_proteomics[mayo_proteomics["Protein IDs"].notna() | mayo_proteomics["Protein IDs"].str.contains("_HUMAN")]
mayo_proteomics["Protein IDs"] = mayo_proteomics["Protein IDs"].apply(lambda x: x.split("_")[0])
mayo_proteomics.set_index("Protein IDs", inplace = True) 
mayo_proteomics = mayo_proteomics.T 
# Ensure numeric & treat zeros as missing (common in proteomics)
mayo_proteomics = mayo_proteomics.apply(pd.to_numeric, errors="coerce").replace(0, np.nan)

# See which protein columns are duplicated
dup_cols = mayo_proteomics.columns[mayo_proteomics.columns.duplicated()].unique()
print(f"{len(dup_cols)} duplicated protein columns")

# Collapse duplicates by **median** per sample (row-wise)
mayo_proteomics = mayo_proteomics.T.groupby(level=0).median().T    # groups by column name 
mayo_proteomics.columns.name = None 

mayo_proteomics = mayo_proteomics[mayo_proteomics.index.str.contains("LFQ intensity")]
mayo_proteomics.index = (
    mayo_proteomics.index.astype(str)
    .str.replace(r"^LFQ intensity\s*mayo_", "", regex=True)
) 
mayo_proteomics = mayo_proteomics.loc[-
    mayo_proteomics.index.to_series().str.contains(r"_egis_|_mgis_", na=False)
]
mayo_proteomics.index = mayo_proteomics.index.to_series().str[:-3]
# select mappable samples
mappable_samples = [sample for sample in mayo_proteomics.index if sample in MayKeysMap]
mayo_proteomics = mayo_proteomics.loc[mappable_samples] 
mayo_proteomics.index = [specimenID_to_individualID_MAYO[sample] for sample in mayo_proteomics.index]
mayo_proteomics.head()

289 duplicated protein columns


,1433B,1433E,1433F,1433G,1433S,1433T,1433Z,1A02,1A31,1A33,...,ZNT9,ZO1,ZO2,ZRAB2,ZSC18,ZSWM3,ZW10,ZY11B,ZYX,ZZEF1
7095,1.085800e+10,2.491900e+10,7.736200e+09,3.961200e+10,7.201400e+09,6.349400e+09,4.770000e+10,211540000.0,NaN,NaN,...,292430000.0,561190000.0,166200000.0,59357000.0,NaN,NaN,NaN,61857000.0,259780000.0,50863000.0
1019,1.058700e+10,2.988400e+10,1.232600e+10,3.494800e+10,6.870100e+09,6.237700e+09,7.214400e+10,172910000.0,NaN,NaN,...,247070000.0,553800000.0,240600000.0,47522000.0,NaN,NaN,NaN,68105000.0,210060000.0,140630000.0
1137,1.433000e+10,2.733800e+10,1.293100e+10,4.028100e+10,8.387500e+09,5.884700e+09,7.983600e+10,NaN,NaN,NaN,...,310630000.0,290470000.0,101310000.0,42914000.0,NaN,NaN,NaN,86295000.0,273190000.0,69372000.0
1036,1.584100e+10,2.969900e+10,1.356900e+10,4.199400e+10,9.896400e+09,6.629400e+09,8.981800e+10,148020000.0,NaN,NaN,...,250340000.0,336600000.0,175060000.0,53249000.0,NaN,NaN,NaN,63842000.0,162740000.0,100630000.0
1010,1.258300e+10,3.040000e+10,1.339100e+10,4.056900e+10,7.966500e+09,6.915900e+09,8.704200e+10,162290000.0,NaN,NaN,...,217430000.0,595110000.0,429380000.0,52718000.0,NaN,NaN,22853000.0,61592000.0,215560000.0,59480000.0


In [121]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer

X = mayo_proteomics.copy()

# (optional) if zeros mean “not quantified”, treat them as missing
X = X.apply(pd.to_numeric, errors="coerce").replace(0, np.nan)

# 1) drop columns with >30% missing
missing_frac = X.isna().mean(axis=0)          # per column (sample)
keep_cols = missing_frac <= 0.30
dropped_samples = missing_frac.index[~keep_cols].tolist()
X = X.loc[:, keep_cols]

# 2) KNN impute across samples
#    Transpose so rows=samples, cols=proteins; then transpose back
imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean")
Xt = X.T
Xt_imp = pd.DataFrame(imputer.fit_transform(Xt), index=Xt.index, columns=Xt.columns)
mayo_proteomics = Xt_imp.T

print(f"Dropped {len(dropped_samples)} samples with >30% missing.")
print(mayo_proteomics.shape)
mayo_proteomics.head()

Dropped 2436 samples with >30% missing.
(198, 3607)


,1433B,1433E,1433F,1433G,1433S,1433T,1433Z,1A02,1B55,2A5A,...,ZN501,ZNT1,ZNT3,ZNT9,ZO1,ZO2,ZRAB2,ZY11B,ZYX,ZZEF1
7095,1.085800e+10,2.491900e+10,7.736200e+09,3.961200e+10,7.201400e+09,6.349400e+09,4.770000e+10,2.115400e+08,65583000.0,90081000.0,...,1.039700e+08,99774000.0,122950000.0,292430000.0,561190000.0,166200000.0,59357000.0,61857000.0,259780000.0,50863000.0
1019,1.058700e+10,2.988400e+10,1.232600e+10,3.494800e+10,6.870100e+09,6.237700e+09,7.214400e+10,1.729100e+08,110220000.0,70390000.0,...,1.405099e+08,149340000.0,291090000.0,247070000.0,553800000.0,240600000.0,47522000.0,68105000.0,210060000.0,140630000.0
1137,1.433000e+10,2.733800e+10,1.293100e+10,4.028100e+10,8.387500e+09,5.884700e+09,7.983600e+10,1.618909e+08,44378000.0,72750000.0,...,1.459400e+08,110840000.0,552900000.0,310630000.0,290470000.0,101310000.0,42914000.0,86295000.0,273190000.0,69372000.0
1036,1.584100e+10,2.969900e+10,1.356900e+10,4.199400e+10,9.896400e+09,6.629400e+09,8.981800e+10,1.480200e+08,48861000.0,73366000.0,...,1.599300e+08,103680000.0,407440000.0,250340000.0,336600000.0,175060000.0,53249000.0,63842000.0,162740000.0,100630000.0
1010,1.258300e+10,3.040000e+10,1.339100e+10,4.056900e+10,7.966500e+09,6.915900e+09,8.704200e+10,1.622900e+08,43782000.0,87291000.0,...,2.667400e+07,126610000.0,260460000.0,217430000.0,595110000.0,429380000.0,52718000.0,61592000.0,215560000.0,59480000.0


#### 2.2. Metabolomics Data

In [122]:
mayo_metabolon = pd.read_excel("../downloads/MayoRNASeq/mayo_all_metabolon_raw.XLSX", sheet_name = "OrigScale")
mayo_metabolon = mayo_metabolon.iloc[4:,:]
mayo_metabolon.columns = mayo_metabolon.iloc[0,:]
mayo_metabolon.set_index("BIOCHEMICAL", inplace = True) 
mayo_metabolon = mayo_metabolon.iloc[1:,12:].T 
mayo_metabolon.columns.name = None
mayo_metabolon.index.name = None 
mayo_metabolon.index = [specimenID_to_individualID_MAYO[sample] for sample in mayo_metabolon.index]
mayo_metabolon.head()

,(14 or 15)-methylpalmitate (a17:0 or i17:0),(16 or 17)-methylstearate (a19:0 or i19:0),(3'-5')-adenylylcytidine,(3'-5')-adenylyluridine,(3'-5')-cytidylyluridine*,(N(1) + N(8))-acetylspermidine,(R)-3-hydroxybutyrylcarnitine,(S)-3-hydroxybutyrylcarnitine,"1,2-dilinoleoyl-GPC (18:2/18:2)","1,2-dilinoleoyl-GPE (18:2/18:2)*",...,X - 25790,X - 25855,X - 25856,X - 25936,X - 25948,X - 25957,X - 25979,X - 25981,X - 25982,X - 25983
11452,9975151,1110083.25,29326.9609,NaN,30609.1074,10720839,698109.4375,4001501.5,6094944,922361.9375,...,3903968,3985164.75,592185.5625,15487756,2048522.375,NaN,NaN,4177947,1020262.4375,89547.6797
11403,7518184.5,1010677,22224.4375,NaN,26050.3633,7233130,664627.0625,1682083.375,4977895,498287.5313,...,3471839.5,5238093.5,823148.875,14907027,1957078.625,NaN,NaN,4067847.75,917023,110351.5078
1114,5776744.5,621827.75,NaN,NaN,NaN,9345200,808244.75,6707128.5,5109592.5,704647.4375,...,5400498,3540357.5,466150.4375,14926786,751405.375,NaN,NaN,4097844.25,1318638,91585.2344
952,7253088,771615.8125,NaN,NaN,NaN,17065880,867421.3125,8483913,5898391,1110842.75,...,NaN,4170168.75,663838.6875,33245080,707274.9375,NaN,NaN,3625782.75,862197.5,90086.2656
1239,2529048,262405.5625,NaN,NaN,NaN,10898745,827128.5625,8207748,6402652.5,704877.3125,...,5041379,4877094,904670.3125,30792766,946949.1875,NaN,27380.9004,3096393,1359939,131384.3125


In [123]:
# take care of missing values
X = mayo_metabolon.copy()

# (optional) if zeros mean “not quantified”, treat them as missing
X = X.apply(pd.to_numeric, errors="coerce").replace(0, np.nan)

# 1) drop columns with >30% missing
missing_frac = X.isna().mean(axis=0)          # per column (sample)
keep_cols = missing_frac <= 0.30
dropped_samples = missing_frac.index[~keep_cols].tolist()
X = X.loc[:, keep_cols]

# 2) KNN impute across samples
#    Transpose so rows=samples, cols=proteins; then transpose back
imputer = KNNImputer(n_neighbors=5, weights="distance", metric="nan_euclidean")
Xt = X.T
Xt_imp = pd.DataFrame(imputer.fit_transform(Xt), index=Xt.index, columns=Xt.columns)
mayo_metabolon = Xt_imp.T

print(f"Dropped {len(dropped_samples)} samples with >30% missing.")
print(mayo_metabolon.shape)
mayo_metabolon.head()

Dropped 157 samples with >30% missing.
(196, 670)


,(14 or 15)-methylpalmitate (a17:0 or i17:0),(16 or 17)-methylstearate (a19:0 or i19:0),(N(1) + N(8))-acetylspermidine,(R)-3-hydroxybutyrylcarnitine,(S)-3-hydroxybutyrylcarnitine,"1,2-dilinoleoyl-GPC (18:2/18:2)","1,2-dilinoleoyl-GPE (18:2/18:2)*","1,2-dioleoyl-GPC (18:1/18:1)","1,2-dioleoyl-GPE (18:1/18:1)","1,2-dioleoyl-GPS (18:1/18:1)",...,X - 25422,X - 25790,X - 25855,X - 25856,X - 25936,X - 25948,X - 25979,X - 25981,X - 25982,X - 25983
11452,9975151.0,1.110083e+06,10720839.0,698109.4375,4001501.500,6094944.0,9.223619e+05,146137728.0,186126976.0,114999992.0,...,1.719939e+06,3.903968e+06,3985164.75,592185.5625,15487756.0,2.048522e+06,44323.195897,4177947.00,1.020262e+06,89547.6797
11403,7518184.5,1.010677e+06,7233130.0,664627.0625,1682083.375,4977895.0,4.982875e+05,217908768.0,215385408.0,135376768.0,...,1.010281e+06,3.471840e+06,5238093.50,823148.8750,14907027.0,1.957079e+06,51536.462672,4067847.75,9.170230e+05,110351.5078
1114,5776744.5,6.218278e+05,9345200.0,808244.7500,6707128.500,5109592.5,7.046474e+05,106939896.0,135198144.0,91116824.0,...,1.506226e+06,5.400498e+06,3540357.50,466150.4375,14926786.0,7.514054e+05,42114.799605,4097844.25,1.318638e+06,91585.2344
952,7253088.0,7.716158e+05,17065880.0,867421.3125,8483913.000,5898391.0,1.110843e+06,150668960.0,180041584.0,122730864.0,...,2.460283e+06,3.688862e+06,4170168.75,663838.6875,33245080.0,7.072749e+05,47967.237195,3625782.75,8.621975e+05,90086.2656
1239,2529048.0,2.624056e+05,10898745.0,827128.5625,8207748.000,6402652.5,7.048773e+05,113458200.0,146622976.0,96485160.0,...,1.447545e+06,5.041379e+06,4877094.00,904670.3125,30792766.0,9.469492e+05,27380.900400,3096393.00,1.359939e+06,131384.3125


#### 2.3. Transcriptomics Data

In [126]:
mayo_gene_expression = pd.read_csv("../downloads/MayoRNASeq/MayoRNAseq_RNAseq_TCX_geneCounts_normalized.tsv", sep = "\t")
mayo_gene_expression.set_index("ensembl_id", inplace = True) 
mayo_gene_expression = mayo_gene_expression.T 
mayo_gene_expression.columns.name = None 
mayo_gene_expression.index = [specimenID_to_individualID_MAYO[sample] for sample in mayo_gene_expression.index]

In [59]:
import mygene
mg = mygene.MyGeneInfo()
ensembl_ids = [id.split(".")[0] for id in mayo_gene_expression.columns]

In [61]:
# Query the Ensembl IDs
results = mg.querymany(ensembl_ids, scopes='ensembl.gene', fields='symbol', species='human')

In [62]:
gene_symbols = []
gene_id_to_symbol = {}
# Print the mapping from Ensembl IDs to gene symbols
for result in results:
    ensembl_id = result.get('query')
    gene_symbol = result.get('symbol', 'N/A')
    gene_id_to_symbol[ensembl_id] = gene_symbol

In [127]:
# mapped ids
mapped_ids = [gene_id for gene_id in mayo_gene_expression.columns if gene_id_to_symbol[gene_id.split(".")[0]] != "N/A" ]
len(mapped_ids)

46619

In [128]:
# filter gene_expression by mapped genes
mayo_gene_expression = mayo_gene_expression[mapped_ids]
mayo_gene_expression.columns = [gene_id_to_symbol[gene_id] for gene_id in mayo_gene_expression.columns]

In [129]:
# Remove duplicates while maintaining the original order
new_columns_unique = list(dict.fromkeys(mayo_gene_expression.columns))
len(new_columns_unique)

42374

In [130]:
mayo_gene_expression = mayo_gene_expression[new_columns_unique]
mayo_gene_expression = mayo_gene_expression.loc[:, ~mayo_gene_expression.columns.duplicated()]
mayo_gene_expression.head()

,TSPAN6,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,...,FRG1KP,ERICD,CTBP2P10,DUX4L25,LINC01726,LINC01101,IGHV1-69D,LOC730668,LOC102724200,LOC105379507
11344,10.6908,0.1249,23.0052,10.8656,6.8691,6.9690,26.8768,18.7838,52.7545,28.1258,...,0.0000,0.0999,0.0,0.0,0.0,0.0000,0.0,0.0500,6.1447,0.0
11316,15.2373,0.0612,17.5626,7.2515,6.4865,10.3112,47.6700,17.8992,49.5976,29.8014,...,0.0000,0.2754,0.0,0.0,0.0,0.0000,0.0,0.1530,5.2015,0.0
11431,13.1779,0.1011,17.0585,8.8324,5.5177,5.0327,24.8399,17.4627,39.0283,27.4472,...,0.0000,0.1213,0.0,0.0,0.0,0.0000,0.0,0.0202,5.7603,0.0
11341,11.0598,0.1372,18.9474,10.4254,6.7045,5.3499,23.1999,17.0098,59.7916,29.8186,...,0.0514,0.1543,0.0,0.0,0.0,0.0171,0.0,0.1029,3.8409,0.0
11289,10.4162,0.0930,16.8334,7.0682,5.6421,3.9371,8.1842,14.6323,50.6861,15.6243,...,0.0000,0.6200,0.0,0.0,0.0,0.0000,0.0,0.0310,10.8502,0.0


In [131]:
# Remove columns where the sum is 0
mayo_gene_expression = mayo_gene_expression.loc[:, (mayo_gene_expression.sum() != 0)]
mayo_gene_expression.head()

,TSPAN6,TNMD,DPM1,SCYL3,FIRRM,FGR,CFH,FUCA2,GCLC,NFYA,...,MTCO1P1,FLJ30679,LINC02887,FRG1KP,ERICD,LINC01726,LINC01101,IGHV1-69D,LOC730668,LOC102724200
11344,10.6908,0.1249,23.0052,10.8656,6.8691,6.9690,26.8768,18.7838,52.7545,28.1258,...,0.0000,0.0000,0.2997,0.0000,0.0999,0.0,0.0000,0.0,0.0500,6.1447
11316,15.2373,0.0612,17.5626,7.2515,6.4865,10.3112,47.6700,17.8992,49.5976,29.8014,...,0.0306,0.1224,0.5201,0.0000,0.2754,0.0,0.0000,0.0,0.1530,5.2015
11431,13.1779,0.1011,17.0585,8.8324,5.5177,5.0327,24.8399,17.4627,39.0283,27.4472,...,0.0808,0.0000,0.5659,0.0000,0.1213,0.0,0.0000,0.0,0.0202,5.7603
11341,11.0598,0.1372,18.9474,10.4254,6.7045,5.3499,23.1999,17.0098,59.7916,29.8186,...,0.1715,0.0343,0.2572,0.0514,0.1543,0.0,0.0171,0.0,0.1029,3.8409
11289,10.4162,0.0930,16.8334,7.0682,5.6421,3.9371,8.1842,14.6323,50.6861,15.6243,...,0.0000,0.0310,0.3100,0.0000,0.6200,0.0,0.0000,0.0,0.0310,10.8502


In [132]:
# check for missing values
mayo_gene_expression.isna().sum().sum()

0

#### 2.4 Select Multi-Omics Samples and Save

In [134]:
## select multi-omics samples
set1 = set(mayo_proteomics.index.to_list()) 
set2 = set(mayo_metabolon.index.to_list())
set3 = set(mayo_gene_expression.index.to_list())
set4 = set(mayo_clinical_data.index.to_list())
selected_samples = list(set1 & set2 & set3 & set4)
len(selected_samples)

97

In [135]:
# save datasets
proteomics_data = pd.merge(mayo_clinical_data.reset_index(), mayo_proteomics.reset_index()).rename(columns = {"index": "Sample ID"})
metabolomics_data = pd.merge(mayo_clinical_data.reset_index(), mayo_metabolon.reset_index()).rename(columns = {"index": "Sample ID"})
gene_expression_data = pd.merge(mayo_clinical_data.reset_index(), mayo_gene_expression.reset_index()).rename(columns = {"index": "Sample ID"})

proteomics_data.to_csv("../data/MayoRNASeq/cleaned/proteomics_data.csv", index = False)
metabolomics_data.to_csv("../data/MayoRNASeq/cleaned/metabolomics_data.csv", index = False)
gene_expression_data.to_csv("../data/MayoRNASeq/cleaned/gene_expression_data.csv", index = False)

### 3. Breast Cancer Dataset 

We will analyze Gene Expression, microRNA and Epigenetics (Methylation) data. The dataset is downloaded from the UCSC Xena resource.

These important files are involved:   
* TCGA.BRCA.sampleMap_BRCA_clinicalMatrix
* TCGA.BRCA.sampleMap_miRNA_HiSeq_gene.gz
* TCGA.BRCA.sampleMap_HiSeqV2.gz
* TCGA.BRCA.sampleMap_HumanMethylation450.gz

#### Load Clinical Data

In [25]:
phenotype_data = pd.read_csv('../downloads/BRCA/TCGA.BRCA.sampleMap_BRCA_clinicalMatrix', sep='\t') 

# Define threshold (e.g., 30% missing allowed)
columns_to_keep = ['sampleID', 'sample_type', 'age_at_initial_pathologic_diagnosis', 'gender', 'pathologic_stage']  

phenotype_data = phenotype_data[columns_to_keep]
phenotype_data.rename(columns = {'sampleID': 'Sample ID', 'sample_type':'Diagnosis', 'age_at_initial_pathologic_diagnosis': 'Age', 	'gender':'Gender', 'pathologic_stage':'Stage'}, inplace = True)
phenotype_data.dropna(inplace = True)  
phenotype_data = phenotype_data[phenotype_data.Diagnosis!= 'Metastatic'] 
phenotype_data['Diagnosis'] = phenotype_data['Diagnosis'].apply(lambda x: 'Tumor' if x == 'Primary Tumor' else 'Control')
phenotype_data.set_index('Sample ID', inplace = True)

In [26]:
phenotype_data = phenotype_data[phenotype_data.Diagnosis.isin(["Tumor","Control"])] 
phenotype_data.shape

(1229, 4)

#### 3.1. MicroRNA Expression Data

In [138]:
# load  TCGA.BRCA.sampleMap/miRNA_HiSeq_gene
microRNA_df = pd.read_csv('../downloads/BRCA/TCGA.BRCA.sampleMap_miRNA_HiSeq_gene.gz', sep='\t', index_col = 0) 
microRNA_df = microRNA_df.T 

In [139]:
import pandas as pd

# Example: set threshold for missing rate
threshold = 0.3  # Drop columns with >30% missing values

# Calculate missing percentage per column
missing_rate = microRNA_df.isna().mean()

# Filter columns to keep
cols_to_keep = missing_rate[missing_rate <= threshold].index

# Create filtered dataframe
microRNA_df_filtered = microRNA_df[cols_to_keep]

print(f"Original shape: {microRNA_df.shape}")
print(f"New shape after dropping columns: {microRNA_df_filtered.shape}")


Original shape: (832, 2238)
New shape after dropping columns: (832, 569)


In [140]:
# Impute missing values
from sklearn.impute import KNNImputer
import pandas as pd
import numpy as np

# Make sure microRNA_df is numeric only
X = microRNA_df_filtered.select_dtypes(include=[np.number]).copy()

imputer = KNNImputer(
    n_neighbors=7,      # tweak (common: 3–15)
    weights="distance", # distance-weighted average of neighbors
    metric="nan_euclidean"
)

X_imputed = pd.DataFrame(
    imputer.fit_transform(X),
    index=X.index,
    columns=X.columns
)

microRNA_df_imputed = X_imputed
microRNA_df_imputed.head()

sample,MIMAT0000765,MIMAT0000764,MIMAT0000761,MIMAT0000760,MIMAT0000763,MIMAT0000762,MIMAT0002891,MIMAT0002890,MIMAT0004761,MIMAT0026738,...,MIMAT0004584,MIMAT0004587,MIMAT0004586,MIMAT0003298,MIMAT0003296,MIMAT0003297,MIMAT0023712,MIMAT0003293,MIMAT0027505,MIMAT0017950
TCGA-OL-A66H-01,2.674235,5.827901,4.187861,3.641958,9.971574,3.343561,0.780511,1.054184,0.923845,2.714203,...,4.348927,0.442400,4.493867,2.018050,0.923824,0.780500,1.054235,1.173747,0.442415,1.884223
TCGA-3C-AALK-01,1.943141,4.864784,3.515156,3.411793,8.132570,3.753572,0.396273,1.683974,0.706868,3.534960,...,3.719410,0.706846,3.534975,2.110927,0.538882,0.962345,0.396273,0.840260,0.559889,1.453494
TCGA-AR-A1AH-01,2.431606,5.897214,4.490978,5.214793,9.097876,3.361545,2.143544,1.301613,0.983073,3.669015,...,5.393030,0.573700,4.491025,2.294690,0.983073,0.832744,0.573700,0.885191,0.573700,1.562367
TCGA-AC-A5EH-01,0.908891,4.595051,3.436870,3.876836,7.166947,2.726150,0.434072,2.281377,1.037850,2.228372,...,4.343935,0.767324,4.405009,1.367194,1.156148,0.830600,2.228353,1.156200,0.499180,1.789668
TCGA-EW-A2FW-01,3.297334,6.562564,4.350379,5.293458,7.608273,4.694851,0.659063,2.493794,2.065193,2.793992,...,6.721365,0.659207,7.335780,4.233147,0.520340,1.009598,2.014415,1.374293,0.195100,1.850345


In [141]:
# map names
mapper = load_json("../artifacts/mirBase_to_mirTARBase.json") 
mappable_mirs = [mir for mir in microRNA_df_imputed.columns if mir in mapper] 
microRNA_df_imputed = microRNA_df_imputed[mappable_mirs]
microRNA_df_imputed.columns = [mapper[mir] for mir in microRNA_df_imputed.columns]
microRNA_df_imputed.head()

,hsa-miR-335-5p,hsa-miR-339-5p,hsa-miR-324-5p,hsa-miR-331-3p,hsa-miR-338-3p,hsa-miR-324-3p,hsa-miR-18a-3p,hsa-miR-299-5p,hsa-miR-483-5p,hsa-miR-1287-3p,...,hsa-let-7i-3p,hsa-let-7g-3p,hsa-miR-23b-5p,hsa-miR-15b-3p,hsa-miR-629-3p,hsa-miR-627-5p,hsa-miR-628-3p,hsa-miR-624-5p,hsa-miR-6802-3p,hsa-miR-2355-3p
TCGA-OL-A66H-01,2.674235,5.827901,4.187861,3.641958,9.971574,3.343561,0.780511,1.054184,0.923845,2.714203,...,5.443786,4.348927,0.442400,4.493867,2.018050,0.923824,0.780500,1.173747,0.442415,1.884223
TCGA-3C-AALK-01,1.943141,4.864784,3.515156,3.411793,8.132570,3.753572,0.396273,1.683974,0.706868,3.534960,...,4.963132,3.719410,0.706846,3.534975,2.110927,0.538882,0.962345,0.840260,0.559889,1.453494
TCGA-AR-A1AH-01,2.431606,5.897214,4.490978,5.214793,9.097876,3.361545,2.143544,1.301613,0.983073,3.669015,...,5.505855,5.393030,0.573700,4.491025,2.294690,0.983073,0.832744,0.885191,0.573700,1.562367
TCGA-AC-A5EH-01,0.908891,4.595051,3.436870,3.876836,7.166947,2.726150,0.434072,2.281377,1.037850,2.228372,...,6.035477,4.343935,0.767324,4.405009,1.367194,1.156148,0.830600,1.156200,0.499180,1.789668
TCGA-EW-A2FW-01,3.297334,6.562564,4.350379,5.293458,7.608273,4.694851,0.659063,2.493794,2.065193,2.793992,...,6.612487,6.721365,0.659207,7.335780,4.233147,0.520340,1.009598,1.374293,0.195100,1.850345


#### 3.2. Gene Expression Data

In [142]:
# load  TCGA.BRCA.sampleMap/HiSeqV2 (RNASeq)
mRNA_df = pd.read_csv('../downloads/BRCA/TCGA.BRCA.sampleMap_HiSeqV2.gz', sep='\t', index_col = 0) 
mRNA_df = mRNA_df.T
mRNA_df.head()

sample,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,REM1,MTVR2,RTN4RL2,...,TULP2,NPY5R,GNGT2,GNGT1,TULP3,PTRF,BCL6B,GSTK1,SELP,SELS
TCGA-AR-A5QQ-01,9.5074,1.5787,0.0000,11.3676,11.1292,9.9722,11.5966,3.2396,0.0000,3.5764,...,0.5819,0.0000,5.3307,0.5819,9.1928,13.8808,7.3830,11.4289,7.8456,10.7384
TCGA-D8-A1JA-01,7.4346,3.6607,0.6245,11.9181,13.5273,10.8702,12.3048,2.5547,0.0000,6.0854,...,1.0589,0.0000,3.1017,3.7793,9.8340,10.7066,7.2467,9.1673,0.0000,10.1225
TCGA-BH-A0BQ-01,9.3216,2.7224,0.5526,11.9665,11.4105,10.4406,12.8186,4.7115,0.0000,5.8329,...,0.0000,2.7224,4.2976,0.0000,9.2304,12.9973,8.3374,11.3400,8.3765,9.9367
TCGA-BH-A0BT-01,9.0198,1.3414,0.0000,13.1881,11.0911,10.4244,12.6427,2.7553,0.0000,4.6308,...,0.0000,2.9823,4.3067,0.0000,8.7816,12.3298,7.7036,11.3025,7.0585,9.3784
TCGA-A8-A06X-01,9.6417,0.5819,0.0000,12.0036,11.2545,10.1480,12.6622,4.2765,1.8007,4.4505,...,1.3163,0.5819,3.5764,0.0000,9.3024,11.3338,7.7186,10.6898,7.0992,11.1740


In [143]:
# check missing values
mRNA_df.isna().sum().sum()

0

#### 3.3. DNA Methylation Data

In [144]:
# load  TCGA.BRCA.sampleMap/HumanMethylation450
meth_df = pd.read_csv('../downloads/BRCA/TCGA.BRCA.sampleMap_HumanMethylation450.gz', sep='\t', index_col = 0) 
meth_df.shape

(485577, 888)

In [145]:
meth_df.dropna(inplace = True)
meth_df = meth_df.T 
meth_df.head()

sample,cg13332474,cg00651829,cg17027195,cg03050183,cg04244851,cg19669385,cg04244855,cg17689707,cg04244857,cg02434381,...,cg19358568,cg27295654,cg03116837,cg15678817,cg14483317,cg11692435,cg10230711,cg16651827,cg18138552,cg07883722
TCGA-OL-A66H-01,0.0192,0.0179,0.0367,0.0748,0.9398,0.3647,0.9292,0.0158,0.4082,0.0557,...,0.9409,0.7442,0.3527,0.6350,0.0410,0.8846,0.0421,0.7686,0.0320,0.9610
TCGA-3C-AALK-01,0.2032,0.2890,0.0750,0.1251,0.8894,0.8650,0.9008,0.5109,0.7731,0.0727,...,0.9156,0.8053,0.8653,0.7054,0.0479,0.9722,0.0693,0.7614,0.0393,0.9466
TCGA-AC-A5EH-01,0.3003,0.0892,0.0333,0.6269,0.7533,0.5664,0.7468,0.2516,0.4916,0.0825,...,0.6374,0.8708,0.7232,0.7069,0.0306,0.5989,0.0598,0.7281,0.0292,0.9367
TCGA-EW-A2FW-01,0.0287,0.0234,0.0460,0.0947,0.9352,0.9206,0.9478,0.0366,0.4017,0.0998,...,0.9287,0.9060,0.1463,0.8459,0.0692,0.9832,0.1126,0.7433,0.0237,0.9538
TCGA-E9-A1R0-01,0.0382,0.0260,0.0411,0.1677,0.8838,0.5324,0.9512,0.4569,0.6278,0.0720,...,0.7925,0.7750,0.8418,0.6514,0.0541,0.9454,0.0670,0.7220,0.0450,0.9571


In [146]:
# check missing values
meth_df.isna().sum().sum()

0

In [147]:
# select samples with omics intersection 
set1 = set(phenotype_data.index)
set2 = set(microRNA_df_imputed.index)
set3 = set(mRNA_df.index)
set4 = set(meth_df.index) 

multiomics_samples = list(set1 & set2 & set3 & set4)
len(multiomics_samples)

666

In [148]:
phenotype_data.loc[multiomics_samples].Diagnosis.value_counts()

Diagnosis
Tumor      615
Control     51
Name: count, dtype: int64

In [158]:
# save datasets
phenotype_data.index.name = None
microRNA_expression_data = pd.merge(phenotype_data.reset_index(), microRNA_df_imputed.reset_index()).rename(columns = {"index": "Sample ID"})
dna_methylation_data = pd.merge(phenotype_data.reset_index(), meth_df.reset_index()).rename(columns = {"index": "Sample ID"})
gene_expression_data = pd.merge(phenotype_data.reset_index(), mRNA_df.reset_index()).rename(columns = {"index": "Sample ID"})

microRNA_expression_data.to_csv("../data/BRCA/cleaned/miRNA_data.csv", index = False)
dna_methylation_data.to_csv("../data/BRCA/cleaned/dna_methylation_data.csv", index = False)
gene_expression_data.to_csv("../data/BRCA/cleaned/gene_expression_data.csv", index = False)